# MLflow Evaluation Demo: Multi-Turn Conversation with Session-Level Judges

This notebook demonstrates MLflow 3.7's evaluation capabilities by showing how to:
1. Set up session-level judges directly in the notebook
2. Use `mlflow.genai.evaluate()` API to evaluate conversations
3. Extract and interpret evaluation results

**Key Difference from v1**: This notebook showcases MLflow's evaluation methods (`make_judge`, `search_traces`, `genai.evaluate`) by keeping evaluation logic in the notebook rather than hidden in the agent class.

## What You'll Learn

- How to create session-level judges with `{{ conversation }}` template
- How to call `mlflow.genai.evaluate()` directly
- How to extract results from evaluation DataFrames
- Best practices for multi-turn conversation evaluation

---

## Setup: Import Dependencies

In [1]:
from genai.common.config import AgentConfig
from genai.agents.multi_turn.customer_support_agent_simple import CustomerSupportAgentSimple
from genai.agents.multi_turn.scenarios import get_scenario_printer_troubleshooting
from genai.agents.multi_turn.prompts import (
    get_coherence_judge_instructions,
    get_context_retention_judge_instructions,
    map_retention_to_score
)
from genai.common.mlflow_config import setup_mlflow_tracking
import mlflow
from mlflow.genai.judges import make_judge
from typing_extensions import Literal
import os
from pathlib import Path

# Load environment variables
try:
    from dotenv import load_dotenv
    env_file = Path(".env")
    if env_file.exists():
        load_dotenv(env_file)
        print(f"✓ Loaded environment variables from {env_file.absolute()}")
    else:
        print(f"ℹ️  No .env file found. Set environment variables manually.")
except ImportError:
    print("ℹ️  python-dotenv not installed. Set environment variables manually.")

## Convenience for display
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

✓ Loaded environment variables from /Users/danny.chiao/personal/mlflow-misc/genai/agents/multi_turn/.env


## Configuration

In [2]:
# Provider and model configuration
PROVIDER = "databricks"  # or "openai"
AGENT_MODEL = "databricks-gpt-5"
JUDGE_MODEL = "databricks-gemini-2-5-flash"
TEMPERATURE = 1.0
EXPERIMENT_NAME = "customer-support-v2-demo"

print("✓ Configuration:")
print(f"  Provider: {PROVIDER}")
print(f"  Agent Model: {AGENT_MODEL}")
print(f"  Judge Model: {JUDGE_MODEL}")

✓ Configuration:
  Provider: databricks
  Agent Model: databricks-gpt-5
  Judge Model: databricks-gemini-2-5-flash


## Step 1: Setup MLflow Tracking

In [3]:
mlflow.openai.autolog()
using_databricks_mlflow = False
if using_databricks_mlflow:
    mlflow.set_tracking_uri("databricks")
    EXPERIMENT_NAME = f"/Users/danny.chiao@databricks.com/{EXPERIMENT_NAME}"
    mlflow.set_experiment(EXPERIMENT_NAME)
else:
    mlflow.set_tracking_uri(None)
    mlflow.set_experiment(EXPERIMENT_NAME)

2025/12/09 11:07:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/09 11:07:21 INFO mlflow.store.db.utils: Updating database tables
2025/12/09 11:07:21 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/09 11:07:21 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/09 11:07:21 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2025/12/09 11:07:21 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2025/12/09 11:07:21 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2025/12/09 11:07:21 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2025/12/09 11:07:21 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2025/12/09 11:07:21 INFO alembic.runtime.migration: Running 

## Step 2: Initialize Agent (Conversation-Only)

Note: This agent handles conversations only. We'll set up evaluation separately.

In [4]:
config = AgentConfig(
    model=AGENT_MODEL,
    provider=PROVIDER,
    temperature=TEMPERATURE,
    mlflow_experiment=EXPERIMENT_NAME
)

agent = CustomerSupportAgentSimple(config)

print("✓ Agent initialized (conversation-only)")
print(f"  Provider: {config.provider}")
print(f"  Model: {config.model}")

✓ Agent initialized (conversation-only)
  Provider: databricks
  Model: databricks-gpt-5


## Step 3: Run Conversation Scenario

In [5]:
# Load printer troubleshooting scenario
scenario = get_scenario_printer_troubleshooting()

print(f"Scenario: {scenario['name']}")
print(f"Turns: {len(scenario['messages'])}")
print(f"Session ID: {scenario['session_id']}")

Scenario: Printer Troubleshooting
Turns: 4
Session ID: session-printer-001


In [6]:
# Run the conversation
# Under the hood, it tags messages with a session id (mlflow.update_current_trace(metadata={"mlflow.trace.session": session_id}))
# Then it calls into OpenAI for responses, but sends simulated pre-canned user responses
with mlflow.start_run(run_name=scenario['name']) as run:
    conv_result = agent.run_conversation(
        messages=scenario['messages'],
        session_id=scenario['session_id']
    )
    run_id = run.info.run_id

print(f"\n✓ Conversation completed")
print(f"  Run ID: {run_id}")
print(f"  Turns: {conv_result['turns']}")
print(f"Now run `mlflow ui` from this directory to see the experiment in http://127.0.0.1:5000/#/experiments/1/chat-sessions/{scenario['session_id']}")


Running 4-turn conversation (Session: session-printer-001)

Turn 1/4
  User: My HP LaserJet 3000 won't turn on at all. The power light doesn't come on.
  Agent: Sorry you’re having trouble. Let’s narrow this down:

1) Power source
- Try a different wall outlet (direct to wall, no surge protector/UPS).
- Check the circuit breaker/outlet with another device.

2) Cables and switch
- Reseat the power cord at printer and wall; inspect for damage.
- Ensure the rear power switch (if present) is set to On.
- If you have another compatible power cord, try it.

3) Power module
- Unplug for 60 seconds, hold the printer’s power button 10 seconds (if present), then plug back in.

Questions:
- Do you hear any fans/clicks or see any brief LEDs?
- Are you using a surge protector?
- Any recent power surges or moves?

Turn 2/4
  User: Yes, I've checked the power cable and it's plugged in securely to both the printer and wall outlet.
  Agent: Thanks for checking that. Let’s continue step by step:

1) Ou

---

# 🎯 Evaluation Showcase: MLflow Methods

Now we'll demonstrate MLflow's evaluation capabilities step-by-step.

## Step 4: Create Session-Level Judges with `make_judge()`

This is where we showcase MLflow's judge creation API.

In [7]:
# First, let's see what the judges will evaluate
print("="*70)
print("COHERENCE JUDGE INSTRUCTIONS")
print("="*70)
print(get_coherence_judge_instructions())
print("\n" + "="*70)
print("CONTEXT RETENTION JUDGE INSTRUCTIONS")
print("="*70)
print(get_context_retention_judge_instructions())
print("\n" + "="*70)
print("\n💡 Both instructions contain {{ conversation }} - this makes them session-level!")

COHERENCE JUDGE INSTRUCTIONS
You are evaluating the coherence of a multi-turn customer support conversation.

Conversation to evaluate:
{{ conversation }}

Evaluate whether the conversation flows logically:
- Does the agent maintain context across turns?
- Are responses relevant to previous messages?
- Does the conversation follow a logical progression?
- Are there any contradictions or confusing jumps?

Provide your evaluation as:

- Value: True if the conversation is coherent and flows naturally, False if there are significant coherence issues.

- Rationale: Explain your reasoning in 2-3 sentences. Consider:
  - Context maintenance: Agent remembers what user said earlier
  - Logical flow: Each turn builds on previous turns
  - Relevance: Responses address the actual questions/issues raised
  - Consistency: No contradictions in advice or information


CONTEXT RETENTION JUDGE INSTRUCTIONS
You are evaluating context retention in a multi-turn customer support conversation.

Conversation 

In [8]:
# Configure judge model URI
if PROVIDER == "databricks":
    os.environ["OPENAI_API_KEY"] = os.environ.get("DATABRICKS_TOKEN", "")
    os.environ["OPENAI_API_BASE"] = f"{config.databricks_host}/serving-endpoints"
    judge_model_uri = f"openai:/{JUDGE_MODEL}"
else:
    judge_model_uri = JUDGE_MODEL

print(f"Judge model URI: {judge_model_uri}")

Judge model URI: openai:/databricks-gemini-2-5-flash


In [9]:
# Create judges using mlflow.genai.judges.make_judge()
print("Creating judges...\n")

coherence_judge = make_judge(
    name="conversation_coherence",
    model=judge_model_uri,
    instructions=get_coherence_judge_instructions(),
    feedback_value_type=bool
)

context_judge = make_judge(
    name="context_retention",
    model=judge_model_uri,
    instructions=get_context_retention_judge_instructions(),
    feedback_value_type=Literal["excellent", "good", "fair", "poor"]
)

print("✓ Judges created")
print(f"  Coherence judge is session-level: {coherence_judge.is_session_level_scorer}")
print(f"  Context judge is session-level: {context_judge.is_session_level_scorer}")

Creating judges...

✓ Judges created
  Coherence judge is session-level: True
  Context judge is session-level: True


## Step 5: Search for Traces with `mlflow.search_traces()`

In [10]:
# Get experiment and search for traces
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

session_traces = mlflow.search_traces(
    experiment_ids=[experiment.experiment_id],
    filter_string=f"metadata.`mlflow.trace.session` = '{scenario['session_id']}'"
)

print(f"✓ Found {len(session_traces)} traces")
print(f"  Session ID: {scenario['session_id']}")
print(f"  Each trace = 1 conversation turn")
display(session_traces[["request_time", "request", "response"]].sort_values(by="request_time", ascending=True))

✓ Found 4 traces
  Session ID: session-printer-001
  Each trace = 1 conversation turn


/var/folders/5v/7y2bspmn7fs31bnn_n8pl6v80000gp/T/ipykernel_34499/1991061166.py:4: FutureWarning: Parameter 'experiment_ids' is deprecated. Please use 'locations' instead.
  session_traces = mlflow.search_traces(


,request_time,request,response
3,1765307246775,"{'message': 'My HP LaserJet 3000 won't turn on at all. The power light doesn't come on.', 'session_id': 'session-printer-001'}","Sorry you’re having trouble. Let’s narrow this down:\n\n1) Power source\n- Try a different wall outlet (direct to wall, no surge protector/UPS).\n- Check the circuit breaker/outlet with another device.\n\n2) Cables and switch\n- Reseat the power cord at printer and wall; inspect for damage.\n- Ensure the rear power switch (if present) is set to On.\n- If you have another compatible power cord, try it.\n\n3) Power module\n- Unplug for 60 seconds, hold the printer’s power button 10 seconds (if present), then plug back in.\n\nQuestions:\n- Do you hear any fans/clicks or see any brief LEDs?\n- Are you using a surge protector?\n- Any recent power surges or moves?"
2,1765307250686,"{'message': 'Yes, I've checked the power cable and it's plugged in securely to both the printer and wall outlet.', 'session_id': 'session-printer-001'}","Thanks for checking that. Let’s continue step by step:\n\n1) Outlet test\n- Plug a lamp/phone charger into the same outlet to confirm it has power.\n- If possible, try a different wall outlet on a different circuit (no surge protector/UPS).\n\n2) Power switch/cord\n- Ensure the rear power switch is On.\n- Try a different IEC C13 power cord (same type used by PCs/monitors).\n\n3) Power reset\n- Unplug printer, wait 60 seconds, hold power button (if present) for 10 seconds, then plug directly into wall and try again.\n\nPlease share:\n- Any brief LEDs/fan noises on power-up?\n- Using a surge protector?\n- Any recent power outage or move?"
1,1765307254573,"{'message': 'I tried a different outlet in another room and still nothing. What should I try next?', 'session_id': 'session-printer-001'}","Thanks for testing another outlet. Next steps:\n\n1) Try a different power cord\n- Use a known-good IEC C13 cord (from a PC/monitor). Inspect for burns/kinks.\n\n2) Hard reset\n- Unplug printer.\n- Hold the power button (if present) for 15 seconds.\n- Plug directly into wall and switch On.\n\n3) Check rear power switch and fuse\n- Ensure rear rocker switch is On.\n- Some LaserJets have a fuse near the power inlet—inspect for a blown/loose fuse if accessible.\n\nIf still dead: likely failed power supply/low-voltage power board. Do you hear any clicks/LED flash at all? If none, I can help locate a service center or part options."
0,1765307258384,"{'message': 'Okay, I'll contact HP support for a warranty replacement. Thanks for your help!', 'session_id': 'session-printer-001'}","You’re welcome! Before you call, have these handy to speed things up:\n- Printer model (HP LaserJet 3000) and serial number\n- Purchase date/proof of purchase\n- Steps you’ve already tried (outlet, different room, cord check, hard reset)\n\nIf HP asks, note there are no LEDs/fans/clicks on power-up and it’s plugged directly into a working wall outlet.\n\nIf you need help finding the serial number or a local service center, tell me your region and I’ll guide you."


## Step 6: Evaluate with `mlflow.genai.evaluate()`

**Key MLflow API**: This is where the magic happens!

In [ ]:
print(f"Evaluating conversation...\n")

eval_results = mlflow.genai.evaluate(
    data=session_traces,
    scorers=[coherence_judge, context_judge]
)

print("✓ Evaluation complete")
result_df = eval_results.result_df
print(f"  Result DataFrame shape: {result_df.shape}")
if not using_databricks_mlflow:
    print(f"See results at http://localhost:5000/#/experiments/1/chat-sessions/session-printer-001")
    print(f"See results at http://localhost:5000/#/experiments/1/evaluation-runs?selectedRunUuid={eval_results.run_id}")

2025/12/09 11:07:41 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


Evaluating conversation...



Evaluating:   0%|          | 0/4 [Elapsed: 00:00, Remaining: ?] 

In [ ]:
display(result_df[["assessments"]])

,assessments
0,"[{'assessment_id': 'a-62e93e633935408683c1eb593f4b5a32', 'assessment_name': 'conversation_coherence', 'trace_id': 'tr-ff879075a91edf51416637507a5871c5', 'span_id': '404e9b0f5ebd27ab', 'source': {'source_type': 'LLM_JUDGE', 'source_id': 'openai:/databricks-gemini-2-5-flash'}, 'create_time': '2025-12-09T18:58:16.098Z', 'last_update_time': '2025-12-09T18:58:16.098Z', 'feedback': {'value': True}, 'rationale': 'The conversation flows very logically, with the agent consistently maintaining context by acknowledging the user's previous actions and responses. Each turn builds on the last, providing progressively more detailed troubleshooting steps relevant to the user's 'printer won't turn on' issue. There are no contradictions or confusing jumps, and the agent offers helpful next steps even after the user decides to contact support.', 'metadata': {'mlflow.trace.session': 'session-printer-001', 'mlflow.assessment.sourceRunId': '434cca9ca09941d4baf4756cb7477004'}, 'valid': True}, {'assessment_id': 'a-bb02d006ae1d4bdda28133487846363a', 'assessment_name': 'context_retention', 'trace_id': 'tr-ff879075a91edf51416637507a5871c5', 'span_id': '404e9b0f5ebd27ab', 'source': {'source_type': 'LLM_JUDGE', 'source_id': 'openai:/databricks-gemini-2-5-flash'}, 'create_time': '2025-12-09T18:58:20.846Z', 'last_update_time': '2025-12-09T18:58:20.846Z', 'feedback': {'value': 'excellent'}, 'rationale': 'The agent perfectly recalls all relevant prior context, explicitly acknowledging the user's completed steps (checking the power cable, trying different outlets). It builds on previous exchanges by suggesting new, logical troubleshooting steps without asking for information already provided. The conversation feels connected and progressive, leading to a clear diagnosis and helpful advice for the next steps.', 'metadata': {'mlflow.trace.session': 'session-printer-001', 'mlflow.assessment.sourceRunId': '434cca9ca09941d4baf4756cb7477004'}, 'valid': True}]"
1,[]
2,[]
3,[]


---

## 📊 What Just Happened?

We just demonstrated the complete MLflow evaluation workflow:

1. **`make_judge()`**: Created session-level judges with `{{ conversation }}` template
2. **`mlflow.search_traces()`**: Found all traces for the conversation
3. **`mlflow.genai.evaluate()`**: Evaluated the full conversation (not individual turns)

**Key Insight**: Session-level evaluation assesses the entire conversation holistically, enabling evaluation of:
- Conversation coherence and flow
- Context retention across turns
- Logical progression of the discussion

## 🎯 Try It Yourself

Modify the scenario or create your own!

In [ ]:
def evaluate_custom_scenario(name: str, messages: list, session_id: str = None):
    """
    Run and evaluate a custom conversation scenario.
    """
    import uuid
    
    if session_id is None:
        session_id = f"session-custom-{uuid.uuid4().hex[:8]}"
    
    print(f"\n{'='*70}")
    print(f"Custom Scenario: {name}")
    print(f"{'='*70}\n")
    
    # Run conversation
    with mlflow.start_run(run_name=name) as run:
        conv = agent.run_conversation(messages=messages, session_id=session_id)
        custom_run_id = run.info.run_id
    
    # Search traces
    traces = mlflow.search_traces(
        experiment_ids=[experiment.experiment_id],
        filter_string=f"run_id = '{custom_run_id}'"
    )
    
    # Evaluate
    print("\nEvaluating...\n")
    eval_res = mlflow.genai.evaluate(
        data=traces,
        scorers=[coherence_judge, context_judge]
    )
    
    # Extract
    df = eval_res.result_df
    coh_cols = [col for col in df.columns if 'coherence' in col.lower()]
    ctx_cols = [col for col in df.columns if 'context' in col.lower()]
    
    coh_val = df[[col for col in coh_cols if '/value' in col][0]].iloc[0]
    coh_rat = df[[col for col in coh_cols if '/justification' in col or '/rationale' in col][0]].iloc[0]
    
    ctx_val = df[[col for col in ctx_cols if '/value' in col][0]].iloc[0]
    ctx_rat = df[[col for col in ctx_cols if '/justification' in col or '/rationale' in col][0]].iloc[0]
    ctx_scr = map_retention_to_score(ctx_val)
    
    # Display
    print(f"{'='*70}")
    print(f"Results: {name}")
    print(f"{'='*70}")
    print(f"\n📊 Coherence: {'✓ PASS' if coh_val else '✗ FAIL'}")
    print(f"   {coh_rat}")
    print(f"\n🧠 Context Retention: {str(ctx_val).upper()} ({ctx_scr}/4)")
    print(f"   {ctx_rat}")
    print(f"\n{'='*70}\n")
    
    return {"coherence": coh_val, "context_retention": ctx_val, "context_score": ctx_scr}

# Example:
# result = evaluate_custom_scenario(
#     name="Product Return",
#     messages=[
#         "I want to return a laptop. It's defective.",
#         "Order ORD-9876. The screen flickers.",
#         "Yes, I have the box and accessories.",
#         "Thanks! I'll ship it tomorrow."
#     ]
# )

print("✓ Helper function defined: evaluate_custom_scenario()")

## 📚 Summary: MLflow Evaluation Methods Demonstrated

### Key MLflow APIs Used

1. **`mlflow.genai.judges.make_judge()`**
   - Creates judges with custom instructions
   - Supports `{{ conversation }}` template for session-level evaluation
   - Specify feedback types (bool, categorical, numeric)

2. **`mlflow.search_traces()`**
   - Find traces by experiment, run ID, or filters
   - Groups traces with same session ID
   - Returns trace data for evaluation

3. **`mlflow.genai.evaluate()`**
   - Evaluates data (traces, DataFrames, etc.) with judges
   - Returns evaluation results as DataFrame
   - Handles both session-level and turn-level evaluation

4. **Session Tracking Pattern**
   - `mlflow.update_current_trace(metadata={'mlflow.trace.session': session_id})`
   - Tags each trace with session ID
   - Enables multi-turn conversation evaluation

### Why This Approach?

✅ **Transparency**: See exactly how evaluation works  
✅ **Flexibility**: Easy to customize judges and evaluation logic  
✅ **Best Practices**: Direct MLflow API usage patterns  
✅ **Reproducibility**: Copy cells to your own projects  

### Next Steps

1. Customize judge instructions in `prompts.py`
2. Try different scenarios with `evaluate_custom_scenario()`
3. Explore MLflow UI: `mlflow ui`
4. Apply to your own conversational AI use cases